In [ ]:
import torch
import torchvision
from torch import optim
import os
import gc
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tqdm
from torch.utils.data import Dataset, DataLoader, TensorDataset
from matplotlib import pyplot as plt
import PIL.Image as pilimg
from torch.autograd import Variable
from sklearn.metrics import plot_confusion_matrix, classification_report

Function

In [ ]:
def valid(data_loader, model):
    """ model inference """
    n_predict = 0
    n_correct = 0
    with torch.no_grad():
        for X, Y in tqdm.tqdm(data_loader):
            X, Y = X.cuda(), Y.cuda()
            y_hat = model(X)
            y_hat.argmax()

            _, predicted = torch.max(y_hat, 1)
            
            n_predict += len(predicted)

            predicted = predicted.unsqueeze(dim=1)
            correct = (Y == predicted).squeeze(dim=-1)
            n_correct += int(correct.sum())

    accuracy = n_correct/n_predict
    print(f"Accuracy: {accuracy} ()")

In [ ]:
def test(data_loader, model):
    """ model inference """
    n_predict = 0
    n_correct = 0
    y_pred = []
    y_label = []
    model.load('/content/drive/MyDrive/Data/model/batch32_lr1e-4_epoch95.pt')
    with torch.no_grad():
        for X, Y in tqdm.tqdm(data_loader):
            X, Y = X.cuda(), Y.cuda()
            y_hat = model(X)
            y_hat.argmax()
            
            _, predicted = torch.max(y_hat, 1)
            
            n_predict += len(predicted)
            predicted = predicted.unsqueeze(dim=1)
            correct += (Y == predicted).squeeze(dim=-1)
            n_correct += int(correct.sum())
            
    accuracy = n_correct/n_predict
    print(f"Accuracy: {accuracy} ()")

    print(classification_report(y_label, y_pred))

Model
: based on vggne

In [ ]:
class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()

        self.conv_layer1 = torch.nn.Sequential(   #224*224*3
            torch.nn.Conv2d(1, 64, kernel_size=3, padding=1),   #224x224x64
            torch.nn.ReLU(),
            torch.nn.Conv2d(64, 64, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.conv_layer2 = torch.nn.Sequential(   
            torch.nn.Conv2d(64, 128, kernel_size=3 , padding=1),   #112x112x128
            torch.nn.ReLU(),
            torch.nn.Conv2d(128, 128, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )  

        self.conv_layer3 = torch.nn.Sequential(
            torch.nn.Conv2d(128, 256, kernel_size=3 , padding=1),   #56x56x256
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(256, 256, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.conv_layer4 = torch.nn.Sequential(
            torch.nn.Conv2d(256, 512, kernel_size=3 , padding=1),   #28x28x512
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),
        )

        self.conv_layer5 = torch.nn.Sequential(
            torch.nn.Conv2d(512, 512, kernel_size=3 , padding=1),   #14x14x512
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.Conv2d(512, 512, kernel_size=3, padding=1),
            torch.nn.ReLU(),
            torch.nn.MaxPool2d(kernel_size=2, stride=2),   #-> 7x7x512로 변화
        )

        self.fc6 = torch.nn.Linear(7*7*512, 4096, bias = True)
        torch.nn.init.xavier_uniform_(self.fc6.weight)
        self.layer6 = torch.nn.Sequential(
            self.fc6,
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(4096),
            torch.nn.Dropout(p=0.15)
        )

        self.fc7 = torch.nn.Linear(4096, 4096, bias = True)
        torch.nn.init.xavier_uniform_(self.fc7.weight)
        self.layer7 = torch.nn.Sequential(
            self.fc7,
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(4096),
            torch.nn.Dropout(p=0.15)
        )

        self.fc8 = torch.nn.Linear(4096, 1000, bias = True)
        torch.nn.init.xavier_uniform_(self.fc8.weight)
        self.layer8 = torch.nn.Sequential(
            self.fc8,
            torch.nn.ReLU(),
            torch.nn.BatchNorm1d(1000),
            torch.nn.Dropout(p=0.15)
        )

        self.fc9 = torch.nn.Linear(1000, 20, bias = True)
        torch.nn.init.xavier_uniform_(self.fc9.weight)
    
        for m in self.modules():
            if isinstance(m, torch.nn.Linear):
                torch.nn.init.uniform_(m.weight.data)

    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.conv_layer2(out)
        out = self.conv_layer3(out)
        out = self.conv_layer4(out)
        out = self.conv_layer5(out)
        out = out.view(out.size(0), -1)
        out = self.layer6(out)
        out = self.layer7(out)
        out = self.layer8(out)
        out = self.fc9(out)
        return out


model = Net()
model = model.cuda()

In [ ]:
saved_model_path = ('/content/drive/MyDrive/Data/model/batch32_lr1e-4_epoch95.pt')

model = torch.load(saved_model_path)

In [ ]:
# print(model.state_dict())

Parameter

In [ ]:
model_path = ('/content/drive/MyDrive/Data/model/batch16_lr1e-4_epoch')

plt_path = ('/content/drive/MyDrive/Data/learning_curve/batch16_lr1e-4_3rdtime.png')

data_batch_size = 32

criterion = torch.nn.CrossEntropyLoss().cuda()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)

training_epochs = 10000

Dataset

In [ ]:
# train_x = np.load('/content/drive/MyDrive/Data/Dataset/train_x.npy')
# train_y = np.load('/content/drive/MyDrive/Data/Dataset/train_y.npy')

# valid_x = np.load('/content/drive/MyDrive/Data/Dataset/validation_x.npy')
# valid_y = np.load('/content/drive/MyDrive/Data/Dataset/validation_y.npy')

In [ ]:
# print(train_x.shape, train_y.shape, valid_x.shape, valid_y.shape)

(21916, 1, 224, 224) (21916, 1) (1994, 1, 224, 224) (1994, 1)


In [ ]:
train_x = np.load('/content/drive/MyDrive/Data/Dataset/train_x.npy')
train_y = np.load('/content/drive/MyDrive/Data/Dataset/train_y.npy')

valid_x = np.load('/content/drive/MyDrive/Data/Dataset/validation_x.npy')
valid_y = np.load('/content/drive/MyDrive/Data/Dataset/validation_y.npy')

train_x = torch.from_numpy(train_x).float()
train_y = torch.from_numpy(train_y).long()

valid_x = torch.from_numpy(valid_x).float()
valid_y = torch.from_numpy(valid_y).long()

train_data = torch.utils.data.TensorDataset(train_x, train_y)
valid_data = torch.utils.data.TensorDataset(valid_x, valid_y)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=data_batch_size, shuffle=True)
valid_loader = torch.utils.data.DataLoader(valid_data, batch_size=data_batch_size, shuffle=True)

In [ ]:
test_x = np.load('/content/drive/MyDrive/Data/Dataset/test_x.npy')
test_y = np.load('/content/drive/MyDrive/Data/Dataset/test_y.npy')

In [ ]:
plot = plot_confusion_matrix(model, 
                            test_x, test_y,
                            cmap=plt.cm.Blue)
plt.save('content/drive/MyDrive/Data/confusion_matrix/batch32_lr1e-4_epoch95.png')

Training

In [ ]:
torch.autograd.set_detect_anomaly(True)

losses = []
before_model_path = 'Null'

for epoch in range(training_epochs):
    model.train()
    cost = 0
    n_batches = 0
    for X, y in tqdm.tqdm(train_loader):
        X, y = X.cuda(), y.cuda()
        optimizer.zero_grad()
        y_hat = model(X)
        y = y.squeeze(dim=-1)
        loss = criterion(y_hat, y)
        loss.backward()
        optimizer.step()

        cost += loss.item()
        n_batches += 1

    cost /= n_batches
    losses.append(cost)
    print('[Epoch: {:>4} cost = {:>.9}'.format(epoch+1, cost))
    print("Dev")
    valid(valid_loader, model)
    if min(losses) == cost:
        if os.path.exists(before_model_path):
            os.remove(before_model_path)
        after_model_path = model_path + str(epoch+1) + '_3rdtime.pt'
        before_model_path = after_model_path
        print('save model')
        torch.save(model, after_model_path)

    plt.plot(losses)
    plt.savefig(plt_path)

In [ ]:
# import numpy as np
# data = np.load('/content/drive/MyDrive/Data/Dataset/test_x.npy')
# data_y = np.load('/content/drive/MyDrive/Data/Dataset/test_y.npy')

In [ ]:
# len(data_y)

2026

In [ ]:
# len(data)

2026

In [ ]:
# def test(data_loader, model):
#     """ model inference """
#     n_predict = 0
#     n_correct = 0
#     model.load_state_dict(torch.load("content/drive/MyDrive/Data/Dataset/model3.pt"))
#     with torch.no_grad():
#         for X, Y in tqdm.tqdm(data_loader):
#             X, Y = X.cuda(), Y.cuda()
#             y_hat = model(X)
#             y_hat.argmax()
            
#             _, predicted = torch.max(y_hat, 1)
            
#             n_predict += len(predicted)
#             n_correct += (Y == predicted).sum()
            
#     accuracy = n_correct/n_predict
#     print(f"Accuracy: {accuracy} ()")

In [ ]:
def freeze(model):
    for p in model.parameters():
        p.requires_grad = False

    return model.eval()

In [ ]:
def test(model, loader):
    total_preds = []
    total_labels = []
    
    with torch.no_grad():
        model.cuda()
        for img, label in tqdm.tqdm(loader):
            img = img.cuda()
            label = label.cuda()
            
            pred = model(img)
            total_preds.append(pred)
            total_labels.append(label)
            
        total_preds = torch.cat(total_preds).argmax(dim=1).cpu().numpy()
        total_labels = torch.cat(total_labels).cpu().numpy()

        result = classification_report(total_labels, total_preds, output_dict=True)
        
    return pd.DataFrame(result)

In [ ]:
def check_across_seeds(accs, f1s, result_df, num_classes=20):
    accs = torch.tensor(accs)
    f1s = torch.tensor(f1s)
    
    assert torch.all(torch.abs(accs[1:] - accs[:1]) < 1e-1) and torch.all(torch.abs(f1s[1:] - f1s[:1]) < 1e-1), "test results are not compatible \n{}\n{}".format(accs, f1s)

    print("*** CLASSWISE RESULT ***")
    cwise_result = result_df.loc[['f1-score', 'recall'], [str(i) for i in range(num_classes)]]
    cwise_result = cwise_result.rename(index={'f1-score' : 'f1', 'recall' : 'acc'})
    print(cwise_result)
    
    print("\n*** AVG RESULT ***")
    avg_result = pd.Series({'f1' : result_df.loc['f1-score', 'macro avg'], 'acc' : result_df['accuracy'].values[0]})
    print(avg_result)

In [ ]:
test_x = np.load('/content/drive/MyDrive/Data/Dataset/test_x.npy')
test_y = np.load('/content/drive/MyDrive/Data/Dataset/test_y.npy')

test_x = torch.from_numpy(test_x).float()
test_y = torch.from_numpy(test_y).long()

test_data = torch.utils.data.TensorDataset(test_x, test_y)

test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=True)

In [ ]:
SEEDS = [0, 5, 10]


ACC_LIST = []
F1_LIST = []

for seed in SEEDS:
    gc.collect()
    torch.cuda.empty_cache()

    model = freeze(model)
    RESULT_DF = test(model, test_loader)
    ACC_LIST.append(RESULT_DF['accuracy'].values[0])
    F1_LIST.append(RESULT_DF.loc['f1-score', 'macro avg'])

check_across_seeds(ACC_LIST, F1_LIST, RESULT_DF)

100%|██████████| 63/63 [00:08<00:00,  7.15it/s]


*** CLASSWISE RESULT ***
            0         1         2         3         4         5         6  \
f1   0.648649  0.842491  0.535714  0.852273  0.797203  0.608696  0.617647   
acc  0.750000  0.804196  0.750000  0.815217  0.838235  0.617647  0.608696   

            7         8         9        10        11        12        13  \
f1   0.890701  0.769231  0.489796  0.891344  0.785714  0.765101  0.676923   
acc  0.810089  0.785714  0.610169  0.858156  0.785714  0.802817  0.785714   

           14        15        16        17        18        19  
f1   0.567901  0.835979  0.757895  0.802721  0.588235  0.809524  
acc  0.741935  0.877778  0.727273  0.797297  0.615385  0.850000  

*** AVG RESULT ***
f1     0.726687
acc    0.786359
dtype: float64
